# Implementar una Red Convolucional con PyTorch para clasificación basado en el dataset de agresividad
# Uso de word embeddings como representación de datos


<img src="figs/fig-diagrama-clasificador2.png" width="900">


### 1. **Representar los datos en el modelo de word embeddings seleccionado**:  
   - #### Generalmente, solo se tokeniza para separar adecuadamente las palabras.
   - #### Sin embargo, dependiendo del modelos de word embeddings algunos preprocesamientos puede mejorar la representación.
   - #### Por ejemplo: 
      - ##### tokenizar y separar correctamente las oraciones y palabras
      - ##### convertir a minúsculas
      - ##### quitar acentos (dependiendo de la fuente de datos con la que se generaros los embeddings)
      - ##### quitar números y puntuación 


### 3. **Convertir los datos a vectores densos: word embeddings**:  
   - #### Cada palabra representa un vector, por lo que una oración genera una matriz de vectores

### 4. **Separar los datos para entrenamiento, validación y prueba**:  
   - #### Crear los dataset  con la función train_test_split 
   
### 5. **Definir la arquitectura de la red**:  
   - Definir una red de 1 capa convolucional, una capa maxpooling, una capa completamente conectada como capa de salida

### 6. **Entrenar el modelo**:  
   - Definir los parámetros de las red como: número de épocas, learning_rate, número de neuronas para las capas ocultas, etc.
   
### 7. **Evaluar el modelo**:  
   - Después del entrenamiento, probar la red con las entradas del conjunto de test y evaluar el desempeño con las métricas: Precisión, Recall, F1-score o F1-Measure y Accuracy.
   


# Cargar los datos de entrenamiento

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.model_selection import train_test_split
import fasttext

# colocar la semilla para la generación de números aleatorios para la reproducibilidad de experimentos

random_state = 42
torch.manual_seed(random_state)
np.random.seed(random_state)

#cargar los datos
dataset = pd.read_json("./data/data_aggressiveness_es.json", lines=True)
#conteo de clases
print("Total de ejemplos de entrenamiento")
print(dataset.klass.value_counts())
# Extracción de los textos en arreglos de numpy
X = list(dataset['text'])
# Extracción de las etiquetas o clases de entrenamiento
Y = list(dataset['klass'])



# Cargar el modelo de Word Embeddings

In [ ]:
# Cargar el modelo de word embeddings
ft = fasttext.load_model('/Volumes/data/temp/MX.bin')

# Codificar las etiquetas

In [ ]:
# TODO: Codificar las etiquetas de los datos a una forma categórica numérica: LabelEncoder.

le = LabelEncoder()
# Normalizar las etiquetas a una codificación ordinal para entrada del clasificador
Y_encoded= le.fit_transform(Y)
Y_encoded = list(Y_encoded)
print("Clases:")
print(le.classes_)
print("Clases codificadas:")
print(le.transform(le.classes_))


# Preparar los conjuntos de datos  para el entrenamiento, validación y prueba

In [ ]:
# Dividir el conjunto de datos en conjunto de entrenamiento (80%) y conjunto de pruebas (20%)

X_train, X_test, Y_train, Y_test =  train_test_split(X, Y_encoded, test_size=0.2, stratify=Y_encoded, random_state=42)

# Dividir el conjunto de entrenamiento en:  entrenamiento (90%) y validación (10%)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.1, stratify=Y_train, random_state=42)


In [ ]:
len(X_train), len(Y_train),  len(X_val), len(Y_val), len(X_test), len(Y_test)

# Definición de la arquitectura de la red convolucional para texto

# Arquitectura similar a la propuesta por Yoon Kim

<img src="figs/fig-red_yoon_kim.png" width="900">



Kim, Y. (2014). Convolutional neural networks for sentence classification. En Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP) (pp. 1746-1751). Association for Computational Linguistics. https://doi.org/10.3115/v1/D14-1181

In [ ]:
import fasttext
import numpy as np
import torch
import torch.nn as nn

# Definir el modelo
class NNConv2D(nn.Module):
    def __init__(self, embedding_dim, max_word_length, num_filters, kernel_size, num_classes):
        super().__init__()
        self.in_channel = 1
        self.padding = 0
        self.dilatation = 1
        self.stride = 1
        self.max_word_length = max_word_length
        self.embedding_dim = embedding_dim
        self.windows_size_maxpool = (2, 2)
        self.conv = nn.Conv2d(in_channels=self.in_channel, out_channels=num_filters, kernel_size=kernel_size, stride= self.stride, padding=self.padding)
        self.pool = nn.MaxPool2d(self.windows_size_maxpool )
        self.relu = nn.ReLU()
        
        Hout_conv = (((self.max_word_length + 2*self.padding - self.dilatation * (kernel_size[0] -1 ) -1 )  // self.stride) + 1) 
        Wout_cov = (((embedding_dim +  2*self.padding - self.dilatation * (kernel_size[1] -1 ) -1 )  // self.stride) + 1)

        Hout_pool = ((Hout_conv - (2)) // 2)+1  # Para MaxPool, stride por default es el tamaño de la ventana
        Wout_pool = ((Wout_cov - (2)) // 2)+1  # Para MaxPool,  stride por default es el tamaño de la ventana

        self.fc = nn.Linear(num_filters * Hout_pool * Wout_pool, num_classes)

    def forward(self, x):
        # Forma de entrada de datos
        # (Batch_size, Alto, Ancho)        
        
        # Forma esperada de los datos para CONV2D
        # (N,C,H,W)
        # (Batch_size, Canales, Alto, Ancho)
        # unsqueeze:  Agrega una dimensión en la posición indicada
        x = x.unsqueeze(1)  # Agregar una dimension, en la posición de la dimensión del canal

        # print("después de unsqueeze:",x.shape)
        x = self.conv(x)
        x = self.relu(x)
        # print("después de relu:",x.shape)
        x = self.pool(x)
        # print("después de pool:",x.shape)

        # Aplanar para la capa totalmente conectada
        # x.size(0) Devuelve el tamaño del tensor propio. 
        #           Si se especifica dim, devuelve un entero que contiene el tamaño 
        #           de esa dimensión.
        #           Mantiene la forma del batch, aplana las demás dimensiones
        #
        # x.view:  Devuelve un nuevo tensor con los mismos datos que el tensor propio 
        #          pero de una forma diferente. 
        x = x.view(x.size(0), -1)  

        # print("después de view:",x.shape)
        x = self.fc(x)
        # print("después de fc:",x.shape)
        return x
        

# Forma de Conv2D 

<img src="figs/fig-conv2d_shape.png" width="900">



# Forma de Maxpool2D 

<img src="figs/fig-maxpool2d_shape.png" width="900">

In [ ]:
# Crear dataset personalizado para Dataloader

from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, sentences, labels, ft_model, max_length=100, embedding_dim=300):
        self.sentences = sentences
        self.labels = labels
        self.ft_model = ft_model
        self.max_length = max_length
        self.embedding_dim = embedding_dim

    def sentence_to_embedding(self, sentence):
        if (len(sentence.lower().split()) > self.max_length):
            print("warning oracion mayor, se trunca: ", sentence)
        words = sentence.lower().split()[:self.max_length]

        embedding_matrix = np.zeros((self.max_length, self.embedding_dim))
        for i, word in enumerate(words):
            embedding_matrix[i] = self.ft_model.get_word_vector(word)
        return embedding_matrix

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        embedding = self.sentence_to_embedding(self.sentences[idx])
        label = self.labels[idx]
        return torch.tensor(embedding, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Entrenamiento de la red

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score



torch.manual_seed(42)

# Hiperparámetros
num_filters = 5
kernel_size = (2, 2)
num_classes = 2
batch_size=128
embedding_dim=300
max_word_length = 100

# Crear datasets y dataloaders
train_dataset = TextDataset(X_train, Y_train, ft)
val_dataset = TextDataset(X_val, Y_val, ft)
test_dataset = TextDataset(X_test, Y_test, ft)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


model = NNConv2D(embedding_dim, max_word_length,  num_filters, kernel_size, num_classes)

# Definir pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
epochs = 10
for epoch in range(epochs):
    model.train()
    epoch_loss = 0    
    for embeddings, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()    
    print(f"Época {epoch+1}/{epochs}, Pérdida: {epoch_loss/len(train_loader)}")


    
    # Evaluación
    model.eval()
    y_predicted = torch.empty(0)     
    with torch.no_grad():
        for embeddings, labels in val_loader:
            outputs = model(embeddings)
            y_pred = torch.argmax(outputs, dim=1)
            y_predicted = torch.cat((y_predicted, y_pred)) # Concatenar todas las predicciones de cada batch para al final medir el rendimiento

        print(f"Época {epoch+1}/{epochs}")
        print("P=", precision_score(Y_val, y_predicted, average='macro'))
        print("R=", recall_score(Y_val, y_predicted, average='macro'))
        print("F1=", f1_score(Y_val, y_predicted, average='macro'))
        print("Acc=", accuracy_score(Y_val, y_predicted))


### Evaluación en el conjunto de datos de Test

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# TODO: Evaluación en el conjunto Test

model.eval()
correct = 0
total = 0
y_pred = torch.empty(0) 
with torch.no_grad():
    for embeddings, labels in test_loader:
        outputs = model(embeddings)
        y_pred_test = torch.argmax(outputs, dim=1)
        y_pred = torch.cat((y_pred, y_pred_test))

# TODO: Evaluar el modelo con las predicciones obtenidas y las etiquetas esperadas: 
# classification_report y  matriz de confusión (métricas Precisión, Recall, F1-measaure, Accuracy)


print(confusion_matrix(Y_test, y_pred))
print(classification_report(Y_test, y_pred, digits=4, zero_division='warn'))


### Evaluación de datos nuevos

In [ ]:

x_datos = ["Que basura los put compa\u00f1eros de Pedro", "Se llevo el dinero", "mi app de calendario no sirve"]
# Transformar los datos a vectores densos: word embeddings
y_datos = [-1] * len(x_datos)

datos_nuevos = TextDataset(x_datos, y_datos, ft)
nuevos_loader = DataLoader(datos_nuevos, batch_size=len(x_datos), shuffle=False)

model.eval()
y_pred = torch.empty(0) 
with torch.no_grad():
    for embeddings, labels in nuevos_loader:
        outputs = model(embeddings)
        y_pred_test = torch.argmax(outputs, dim=1)
        y_pred = torch.cat((y_pred, y_pred_test))

y_pred=y_pred.int()

print(y_pred)

print(le.inverse_transform(y_pred))


# Ejercicio 1

In [ ]:
# TODO: Modificar el tamaño de los kernels de convolución y evaluar el rendimiento

# Ejercicio 2

In [ ]:
# TODO: Aumentar  capas completamente conectadas (fully connected) y evaluar el rendimiento

# Ejercicio 3

In [ ]:
# TODO: Aumentar otra capa de convolución y otra capa de maxpooling y evaluar el rendimiento
